In [ ]:
%pwd

In [ ]:
%cd ../..

In [ ]:
%ls

In [ ]:
import glob as glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import tqdm

In [ ]:
minima = [
    -1.0,
    -3.0,
    -6.0,
    -9.103852,
    -12.712062,
    -16.505384,
    -19.821489,
    -24.113360,
    -28.422532,
    -32.765970,
    -37.967600,
    -44.326801,
    -47.845157,
    -52.322627,
    -56.815742,
    -61.317995,
    -66.530949,
    -72.659782,
    -77.1777043,
]

dimensions = 18
k = int(dimensions / 3)
print(minima[k - 2])

# Algorithm Comparison Results

In [ ]:
# Algorithm keep columns
algorithm_columns = [
    'problem_name',
    'algorithm',
    'dimensions',
    'hits',
    'time',
    'f',
]

# Results from comparison algorithms
file_directory = './data-queue-2023-09-24/lennard-jones/*/*'
files = glob.glob(file_directory)
print(files)
algorithm_df = pd.read_parquet(files)

# algorithm_df = algorithm_df[algorithm_columns]
algorithm_df.head()

In [ ]:
algorithm_df.loc[algorithm_df['dimensions'] == 30]

In [ ]:
# Let's get the algorithm compare results!
algorithm_results_df = algorithm_df.groupby(
    ['problem_name', 'algorithm', 'dimensions']
).agg({'hits': ['count', 'mean'], 'time': 'mean'})
algorithm_results_df.columns = [
    '-'.join(column) for column in algorithm_results_df.columns
]
algorithm_results_df = algorithm_results_df.reset_index()

# Need to verify that all problems have the same number of trials in the end
algorithm_results_df = algorithm_results_df.drop(columns=['hits-count'])
algorithm_results_df.pivot_table(
    index='dimensions', columns='algorithm', values='hits-mean'
)

In [ ]:
# Can we visualize the problem?

In [ ]:
import torch


def pairwise_distances(positions: torch.Tensor) -> torch.Tensor:
    # Assume positions has shape [B, 3N] where B is the batch size and N is the number of atoms
    # Reshaping to get individual atoms' positions of shape [B, N, 3]
    positions = positions.view(positions.shape[0], -1, 3)
    # Compute the pairwise differences
    # Subtracting [B, 1, N, 3] from [B, N, 1, 3] gives [B, N, N, 3]
    deltas = positions.unsqueeze(2) - positions.unsqueeze(1)
    # Norm the differences gives [B, N, N]
    distances = torch.norm(deltas, dim=-1)
    return distances


def cluster_potential(positions: torch.Tensor) -> torch.Tensor:
    # Compute the pairwise distances of atoms
    distances = pairwise_distances(positions)

    # Compute the pairwise cost (1 / dist)^12 - (1 / dist)^ 6
    pairwise_cost = (1 / distances).pow(12) - (1 / distances).pow(6.0)

    # Get the upper triangle matrix (ignoring the diagonal)
    ut_pairwise_cost = torch.triu(pairwise_cost, diagonal=1)

    # 4 * Summutation of the upper triangle of pairwise costs gives potential
    potential = 4 * ut_pairwise_cost.sum(dim=(1, 2))
    return potential

In [ ]:
import requests
import tarfile

# Url of tar containing known global minima
url = 'http://doye.chem.ox.ac.uk/jon/structures/LJ/LJ.tar'
# Where to save the tar -- modify as desired
target_path = 'LJ_data.tar'

# Download
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())

# Open file
file = tarfile.open(target_path)

#  By default save the data to the 'LJ_data' folder in the local directory
data_path = f'./{target_path.replace(".tar", "")}'
file.extractall(data_path)

file.close()

In [ ]:
import matplotlib.pyplot as plt
import pandas

# Lists to track atom counts and potentials
atom_counts = []
global_potentials = []

# Iterate from 3 to 67 atoms -- the number visited in the paper
for n_atoms in range(3, 68):
    # File path is simply the nuimber of atoms
    file_path = f'{data_path}/{n_atoms}'
    # Get the positions as a dataframe
    dataframe = pandas.read_csv(file_path, header=None, delim_whitespace=True)
    # Make a positions tensor -- note that we add an initial dimension as the objective function is vectorised
    positions = torch.Tensor(dataframe.to_numpy()).unsqueeze(0)
    # Get the potential
    potential = cluster_potential(positions)

    # Update lists of atom counts and potentials
    atom_counts.append(n_atoms)
    global_potentials.append(potential.item())

# Simple plot
plt.plot(atom_counts, global_potentials)
plt.xlabel('Number of Atoms $N$')
plt.ylabel('Cluster potential $E$')
plt.show()

# Sanity check on the last one
print(
    f'Potential of N={n_atoms} is computed as {potential.item()} vs. published value -347.252007'
)

In [ ]:
from evotorch import Problem
from evotorch.algorithms import SNES

snes_potentials = []

for n_atoms in range(15, 30):
    print(f'Solving case N={n_atoms} with SNES')

    # Set up the problem in vectorised mode
    problem = Problem(
        'min',
        cluster_potential,
        vectorized=True,
        device='cuda:0' if torch.cuda.is_available() else 'cpu',
        dtype=torch.float64,  # Higher precision could be helpful
        solution_length=3 * n_atoms,
        initial_bounds=(-1e-12, 1e-12),  # Taken directly from [2]
        store_solution_stats=True,  # Make sure the problem tracks the best discovered solution, even on GPU
    )

    searcher = SNES(
        problem,
        stdev_init=0.01,  # Taken directly from [2]
        popsize=10 * n_atoms,  # Significantly higher than [2]
        center_learning_rate=0.5,  # Halving value from [2] slows learning
        stdev_learning_rate=0.5,  # Halving value from [2] slows learning
        scale_learning_rate=True,  # Boolean flag means modifying the above learning rates rescales the default
    )
    searcher.run(
        1000 * problem.solution_length
    )  # 2x value used in [2], adjusted for half learning rates

    # Best solution found
    best_potential = problem.status['best_eval']
    # Center is also a good estimate
    center_potential = cluster_potential(searcher.status['center'].cpu().unsqueeze(0))[
        0
    ].item()
    if center_potential < best_potential:
        best_potential = center_potential

    print(f'Found potential {best_potential}')

    snes_potentials.append(best_potential)

In [ ]:
# Simple plot
plt.plot(
    atom_counts[15 : len(snes_potentials)],
    global_potentials[15 : len(snes_potentials)],
    label='Known Optima',
)
plt.plot(
    atom_counts[15 : len(snes_potentials)],
    snes_potentials[15 : len(snes_potentials)],
    label='SNES-discovered Solutions',
)
plt.legend()
plt.xlabel('Number of Atoms $N$')
plt.ylabel('Atom Cluster Potential $E$')
plt.show()

In [8]:
import numpy as np

x = np.random.randn(9)

x = np.reshape(x, (1, -1))
positions = np.reshape(x, (x.shape[0], -1, 3))

# Compute the pairwise differences
deltas = positions[:, :, np.newaxis] - positions[:, np.newaxis, :]

# Norm the differences to get [B, N, N]
distances = np.linalg.norm(deltas, axis=-1) ** 2

# Get the upper triangle matrix (ignoring the diagonal)
distances = np.triu(distances, k=1)

# Provide a mask to eliminate divisions from zero
mask = distances > 0

# Compute the pairwise cost (1 / dist)^12 - (1 / dist)^ 6
result = 1.0 / distances[mask] ** 6 - 1.0 / distances[mask] ** 3
result = 4 * result.sum()
print(result)

-0.6583140746148592


In [9]:
import torch

x = torch.from_numpy(x)
x = x.reshape(1, -1)
positions = x.view(x.shape[0], -1, 3)

# Compute the pairwise differences
# Subtracting [B, 1, N, 3] from [B, N, 1, 3] gives [B, N, N, 3]
deltas = positions.unsqueeze(2) - positions.unsqueeze(1)

# Norm the differences gives [B, N, N]
distances = torch.norm(deltas, dim=-1) ** 2

# Get the upper triangle matrix (ignoring the diagonal)
distances = torch.triu(distances, diagonal=1)

# Provide a mask to eliminate divisions from zero
mask = distances > 0

# Compute the pairwise cost (1 / dist)^12 - (1 / dist)^ 6
result = 1.0 / distances[mask] ** 6 - 1.0 / distances[mask] ** 3
result = 4 * result.sum()

In [10]:
result

tensor(-0.6583, dtype=torch.float64)